# Amazon Review Rating Prediction

In [123]:
import pickle
import re
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import nltk

# import other packages here
import seaborn as sns
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download("stopwords", quiet = True)
nltk.download("wordnet", quiet = True)
english_stopwords = set(nltk.corpus.stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
# !pip3 install pandas-profiling
import pandas_profiling as pp
# import pandas_profiling as pp
# from imblearn.over_sampling import RandomOverSampler

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Dataset Overview

<br>

| Column name 	| Description 	|
|-	|-	|
| marketplace 	| 2 letter country code of the marketplace where the review was written. 	|
| customer_id 	| Random identifier that can be used to aggregate reviews written by a single author. 	|
| review_id 	| The unique ID of the review. 	|
| product_id 	| The unique Product ID the review pertains to. In the multilingual dataset the reviews for the same product in different countries can be grouped by the same product_id. 	|
| product_parent 	| Random identifier that can be used to aggregate reviews for the same product. 	|
| product_title 	| Title of the product. 	|
| product_category 	| Broad product category that can be used to group reviews (also used to group the dataset into coherent parts). 	|
| helpful_votes 	| Number of helpful votes. 	|
| total_votes 	| Number of total votes the review received. 	|
| vine 	| Review was written as part of the Vine program. 	|
| verified_purchase 	| The review is on a verified purchase. 	|
| review_headline 	| The title of the review. 	|
| review_body 	| The review text. 	|
| review_date 	| The date the review was written. 	|
| **star_rating** 	| The 1-5 star rating of the review. 	|

<br>


In [4]:
def read_data():
    return pd.read_csv("amazon_reviews.tsv",
        sep="\t", header=0, error_bad_lines=False, dtype={
        "marketplace": str,
        "customer_id":str,
        "review_id":str,
        "product_id":str,
        "product_parent":str,
        "product_title":str,
        "product_category":str,
        "star_rating":int,
        "helpful_votes":int,
        "total_votes":int,
        "vine":str,
        "verified_purchase":str,
        "review_headline":str,
        "review_body":str,
        "review_date":str
    })

df_reviews = read_data()

## Part A: Data Preprocessing
Since this project only involves review rating prediction, you can remove the following irrelevant features:

* `marketplace`
* `customer_id`
* `review_id` 
* `product_id`
* `product_parent`
* `product category`(since all the products belong to the same category)
* `review_date`

The primary rating predictor will be the review content (i.e., `review_body`). We suggest you perform the following preprocessing tasks on `review_body`:
* Remove all html tags.
* Clean punctuations and special characters.
* Apply lower casing.
* Tokenize the content, then apply stemming or lemmatization to each token.
* Remove English stopwords.

In [5]:
## DATA PREPROCESSING CODE##
df_reviews = df_reviews.drop(['marketplace', 'customer_id', 'review_id', 'review_date',
                              'product_id', 'product_parent', 'product_category'
                             ], axis=1)

In [6]:
def preprocess(text):
    # Remove all html tags.
    text = re.sub("<.*?>", "", text)
    # Clean punctuations and special characters.
    text = re.sub('[%s]' % re.escape(string.punctuation), " ", text)
    # Apply lower casing
    text = text.lower()
    # Tokenize the content, then apply stemming or lemmatization to each token.
    text_list = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)]
    # Remove English stopwords
    text = " ".join([word for word in text_list if word not in english_stopwords])
    return text

def combine_review(text1, text2):
    return preprocess(text1) + " " + preprocess(text2)

In [7]:
df_reviews["review_body"] = df_reviews["review_body"].apply(lambda x: preprocess(x))
df_reviews["review_headline"] = df_reviews["review_headline"].apply(lambda x: preprocess(x))
df_reviews["review"] = df_reviews["review_headline"] + df_reviews["review_body"]

In [8]:
# check for missing value
df_reviews.isnull().sum()

product_title        0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review               0
dtype: int64

In [9]:
raw_df = df_reviews[["review", "star_rating"]]

## Part B: Exploratory Data Analysis
You should investigate the feature and label distributions in your exploratory data analysis. Using Pandas Profiler (see the [Data visualization primer](https://nbviewer.jupyter.org/url/clouddatascience.blob.core.windows.net/m20-foundation-data-science/p2-data-visualization-primer/data_visualization_primer.ipynb)) can provide a good starting point. Pay attention to rows with missing data or outliers, and think about whether to include or remove them. In case the rating distribution is highly imbalanced, you may also consider oversampling the underrepresented classes to get a more balanced sample. However, oversampling should only be performed in part D on the training data only, after you have performed a train-test split.

In [10]:
## YOUR EXPLORATORY DATA ANALYSIS CODE HERE ##
# Class unbalance overview
sns.set(style="ticks")
count = df_reviews['star_rating'].value_counts()
plt.figure(figsize=(10, 6))
ax = sns.barplot(x = count.index, y = count)

In [11]:
sns.catplot(x="vine", y="helpful_votes", kind="violin", hue="verified_purchase", data=df_reviews)

In [12]:
lengths = []
for text in raw_df['review'].tolist():
    lengths.append(len(text.split()))
binwidth = 10
_ = plt.hist(lengths, bins=range(min(lengths), max(lengths) + binwidth, binwidth))

## Part C: Feature Construction
The next task is to construct numerical features for prediction. We recommend starting with frequency-based features, such as term frequency or term frequency - inverse document frequency, based on the review text content.

In [13]:
## YOUR FEATURE CONSTRUCTION CODE HERE ##
tf = TfidfVectorizer(analyzer=str.split, tokenizer=str.split)
X = tf.fit_transform(raw_df['review'])

In [14]:
with open('models/vectorizer.pkl', 'wb') as f:
    pickle.dump(tf, f)

In [15]:
with open('models/vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [16]:
X.shape

(99909, 112808)

In [17]:
y = raw_df["star_rating"] - 1
y.shape

(99909,)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=517)
# make sure to degreade scikit-learn since the installation of imbelearn will automatically upgrade scikit-learn
# which is in conflict of 
# X, y = RandomUnderSampler().fit_resample(X_temp, y_temp)
# X, y = X_temp, y_temp 
# X_train, Y_train = RandomUnderSampler().fit_resample(X_train, Y_train)

## Part D: Model Training and Evaluation
It is now time to train machine learning models to predict review ratings. Your goal in this part is to come up with a model that can achieve an **accuracy score of more than 0.60 on the test data**. Given the ground truth labels `y_true` and your predicted labels `y_pred` (each of which contains integer values from 1 to 5), the grader environment will compute the accuracy score as `sklearn.metrics.accuracy_score(y_true, y_pred)`.


As a starting point, you can try out the classical learning algorithms supported by Sklearn:

* Logistic regression
* Support vector machine
* K-nearest neighbor

You can also experiment with some ensemble methods based on these classifiers, for example [voting classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) or [boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html). Remember to tune the hyperparameters for each learning algorithm to achieve better performance.

**Notes**:
* Similar to Project 5, your trained model will be evaluated on a secret test dataset. Its accuracy on this dataset will be shown in the TPZ Feedback view when you make a submission.
* We recommend that you start with a simple model and complete the rest of the notebook, in case any unexpected issue occurs with model deployment. Once you have finished the entire pipeline, it is simple to come back and continue fine-tuning your model.

In [254]:
## YOUR MODEL TRAINING CODE HERE ##
model = LogisticRegression(multi_class="multinomial", solver='lbfgs', max_iter=110, verbose=1, C=0.99)
# model = VotingClassifier(estimators=[('lr', clf1), ('knn', clf2), ('svm', clf3)], voting='hard')
model.fit(X_train,y_train)
model.score(X_test, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.3s finished


0.6951471034273582

In [255]:
model.fit(X, y)
model.score(X_test, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.7s finished


0.7642705489839248

In [256]:
idx = 1000
feat = tf.transform([raw_df["review"][idx]])
print(raw_df["review"][idx])
print(y[idx])
model.predict(feat)[0]

best 200 ever spenteverything work perfect quality price outstanding even last 2 year wouldnt hesitate buying built speaker plenty loud dont know people think arent loud enough review kid happy
4


4

In [257]:
## YOUR MODEL SAVING CODE HERE ##
filename = 'models/model.pkl'

with open(filename, 'wb') as f:
    pickle.dump(model, f)

## Part E: Model Deployment
Once you have identified your best model, you can fit it on the entire dataset and save all relevant data processing objects to pickle files in the `models` directory, so that you can use it as your deployment directory in the next part.

In [258]:
# service.delete()

In [259]:
import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

In [260]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: final
Azure region: eastus
Subscription id: 5b3a3039-a86c-4c9a-9463-b8a5c02b13c8
Resource group: p5


In [261]:

workspace_model = Model.register(
    workspace = ws, model_path ="models",
    model_name = "deployment_models",
    description = "LogisticRegression, SVM, and KNN"
)

Registering model deployment_models


For the model deployment process, you can reuse most of the provided codes in Project 5 to set up the workspace, ACI instance and configuration. Note that for `myenv.yml`, you should list all the pip packages that you have used in this project. Remember to double check the latest version of each package that is supported by Azure ML; for example, while the latest sklearn version is `0.23.2`, Azure ML [only supports `0.20.3`](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py). In addition, you do not need the `inference-schema` package in this project.

Our grader will send a POST request to your endpoint, where the JSON content will be structured as follows:

```python
request_content = """{
    "n_reviews" : 2,
    "reviews" : [
        {
            "marketplace" : "US",
            "customer_id" : "20422322",
            "review_id" : "R8MEA6IGAHO0B",
            "product_id" : "B00MC4CED8",
            "product_parent" : "217304173",
            "product_title" : "BlackVue",
            "product_category" : "Mobile_Electronics",
            "helpful_votes" : 0,
            "total_votes" : 0,
            "vine" : "N",
            "verified_purchase" : "Y",
            "review_headline" : "Very Happy!	As advertised.",
            "review_body" : "Everything works perfectly, I'm happy",
            "review_date" : "2015-08-31"
        },
        {
            "marketplace" : "US",
            "customer_id" : "15204307",
            "review_id" : "RZGDLDMGMGKLB",
            "product_id" : "B000I3F91O",
            "product_parent" : "90621845",
            "product_title" : "Sangean H205 AM/FM Weather Alert Waterproof",
            "product_category" : "Electronics",
            "helpful_votes" : 0,
            "total_votes" : 0,
            "vine" : "N",
            "verified_purchase" : "Y",
            "review_headline" : "Best shower radio I have ever owned",
            "review_body" : "love shower radio own shower radio best one",
            "review_date" : "2014-11-08"
        },
    ]
}
"""
```

Here `n_reviews` indicates the number of reviews included in the request, and `reviews` is a list of dictionaries, each having the same schema as a row in the provided dataset (but without the `star_rating` attribute, which is what you need to predict).

The expected response is a JSON with the following format:

```python
response_content = """{
    "predictions" : [2, 3]
}
"""
```
where the key `predictions` maps to a list of predicted labels, one for each input review in `request_content`. Keep in mind that this list should be a standard Python list, not a Numpy array or Pandas series, which are not JSON serializable.

**Notes**:
* In your `score.py`, import all Python packages used in your data processing. In `init`, load all relevant objects from pickle files as global variables. Use these objects to transform / predict the test data in the `run` function. See the [Azure documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where) for a complete example of Sklearn model deployment.
* If you use `nltk` in your data processing, note that the Docker environment used for deployment does not have any NLTK data by default. Therefore, you may need to explicitly download them in your `init` function, for example `nltk.download("stopwords")`. If you use the text processing code from Project 3, refer to the `nltk.download` commands provided at the beginning of the Project 3 notebook.

In [262]:
%%writefile ./score.py
## YOUR SCORE.PY CODE HERE ##
import pickle
import re
import requests
import json
import numpy as np
import sklearn
import nltk
import string
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import wordnet
from azureml.core.model import Model


#########################################
#### Primary functions for score.py #####
#########################################
def init():
    global model
    global vectorizer
    model_path = Model.get_model_path("deployment_models") + "/model.pkl"
    vec_path = Model.get_model_path("deployment_models") + "/vectorizer.pkl"
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    with open(vec_path, 'rb') as f:
        vectorizer = pickle.load(f)
    nltk.download('punkt')
    nltk.download("stopwords", quiet = True)
    nltk.download("wordnet", quiet = True)


def run(data):
    return get_predictions(data)


def get_predictions(json_data):
    json_data = json.loads(json_data)
    predictions = []
    data = []
    response = {}
    review_objects = json_data["reviews"]
    for obj in review_objects:
        data.append(combine_review(obj["review_headline"], obj["review_body"]))
    feats = vectorizer.transform(data)
    response["predictions"] = (model.predict(feats) + 1).tolist()
    return response


def preprocess(text):
    # Some init work
    lemmatizer = WordNetLemmatizer()
    english_stopwords = set(nltk.corpus.stopwords.words('english'))
    # Remove all html tags.
    text = re.sub("<.*?>", "", text)
    # Clean punctuations and special characters.
    text = re.sub('[%s]' % re.escape(string.punctuation), " ", text)
    # Apply lower casing
    text = text.lower()
    # Tokenize the content, then apply stemming or lemmatization to each token.
    text_list = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)]
    # Remove English stopwords
    text = " ".join([word for word in text_list if word not in english_stopwords])
    return text


def combine_review(text1, text2):
    return preprocess(text1) + " " + preprocess(text2)

Overwriting ./score.py


In [263]:
!cat score.py

## YOUR SCORE.PY CODE HERE ##
import pickle
import re
import requests
import json
import numpy as np
import sklearn
import nltk
import string
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import wordnet
from azureml.core.model import Model


#########################################
#### Primary functions for score.py #####
#########################################
def init():
    global model
    global vectorizer
    model_path = Model.get_model_path("deployment_models") + "/model.pkl"
    vec_path = Model.get_model_path("deployment_models") + "/vectorizer.pkl"
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    with open(vec_path, 'rb') as f:
        vectorizer = pickle.load(f)
    nltk.download('punkt')
    nltk.download("stopwords", quiet = True)
    nl

In [264]:
## YOUR ENVIRONMENT SETUP HERE ##
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=[
    'azureml-defaults', 'scikit-learn~=0.20.3', 'nltk'
])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults~=1.10.0
  - scikit-learn~=0.20.3
  - nltk
channels:
- anaconda
- conda-forge



In [265]:
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(source_directory ='.',
                                   entry_script="score.py",
                                   environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'data': 'amzn-review',  'method':'machine learning', 'framework':'scikit learn'},
                                               description='deploy')

service = Model.deploy(workspace=ws, 
                           name='aci-deployment-service',
                           models=[workspace_model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

In [266]:
service.wait_for_deployment(show_output = True)
print(service.state)

Running....................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


Once you have obtained a Healthy deployment state, save your `scoring_uri` to a text file called `scoring_uri.txt`. This file will be collected by the submitter when you submit your code.

In [267]:
 print(service.get_logs())

2020-08-09T03:35:46,115328324+00:00 - iot-server/run 
2020-08-09T03:35:46,123462646+00:00 - gunicorn/run 
2020-08-09T03:35:46,123688647+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_5e92db2bce7833c834e83f2394313db9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5e92db2bce7833c834e83f2394313db9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5e92db2bce7833c834e83f2394313db9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5e92db2bce7833c834e83f2394313db9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5e92db2bce7833c834e83f2394313db9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-08-09T03:35:46,143484601+00:00 - rsyslog/run 
EdgeHubC

In [268]:
## YOUR SCORING_URI CODE HERE ##
print(service.scoring_uri)
with open('scoring_uri.txt', 'w') as f:
    f.write(service.scoring_uri)
    print('Save scoring uri successfully!')

http://c5eb8343-fec2-4702-bf59-2d1b0312f60d.eastus.azurecontainer.io/score
Save scoring uri successfully!


We provide the code to send some example POST requests to your deployment URI as follows. You can run it to check that your deployed model is working. Our grader will run the same code to communicate with your endpoint:

In [269]:
example_json = {
    "n_reviews" : 2,
    "reviews" : [
        {
            "marketplace" : "US",
            "customer_id" : "20422322",
            "review_id" : "R8MEA6IGAHO0B",
            "product_id" : "B00MC4CED8",
            "product_parent" : "217304173",
            "product_title" : "BlackVue",
            "product_category" : "Mobile_Electronics",
            "helpful_votes" : 0,
            "total_votes" : 0,
            "vine" : "N",
            "verified_purchase" : "Y",
            "review_headline" : "Very Happy!    As advertised.",
            "review_body" : "Everything works perfectly, I'm happy",
            "review_date" : "2015-08-31"
        },
        {
            "marketplace" : "US",
            "customer_id" : "15204307",
            "review_id" : "RZGDLDMGMGKLB",
            "product_id" : "B000I3F91O",
            "product_parent" : "90621845",
            "product_title" : "Sangean H205 AM/FM Weather Alert Waterproof",
            "product_category" : "Electronics",
            "helpful_votes" : 0,
            "total_votes" : 0,
            "vine" : "N",
            "verified_purchase" : "Y",
            "review_headline" : "Best shower radio I have ever owned",
            "review_body" : "love shower radio own shower radio best one",
            "review_date" : "2014-11-08"
        },
    ]
}

def ping_endpoint(json_data, uri = None):
    if uri is None:
        uri = open("scoring_uri.txt", "r").read()
    headers = {'Content-Type' : 'application/json'}
    response = requests.post(uri, json.dumps(json_data), headers = headers)
    print(response.status_code)
    print(response.text)
    return response.status_code, json.loads(response.text)["predictions"]

Let's test your model on the provided `example_json`. Check that the status code is 200 and the predicted labels are what you expect.

In [270]:
# test the deployed model with 2 samples
status_code, predicted_labels = ping_endpoint(example_json)

200
{"predictions": [5, 5]}


Now you can test your model on a 3000-row subset of the training dataset. Again check that the status code is 200 and the predict labels are a list of 3000 integers. If you encounter any error, check your `service.get_logs()`. One common issue is that the request times out; in other words, your deployed model takes too long to process the test data and generate predictions. To address this, you would need to optimize your data processing steps and redeploy them. Our grader will also send 3000 rows of test data to your endpoint in one single request.

In [271]:
# test the deployed model with a subset of training data
df_training_subset = read_data().iloc[:3000,:]
training_subset_json = {
    "n_reviews" : len(df_training_subset),
    "reviews" : df_training_subset.drop(columns = "star_rating").to_dict("records")
}
status_code, predicted_labels = ping_endpoint(training_subset_json)
print("Training accuracy", accuracy_score(df_training_subset["star_rating"], predicted_labels))

200
{"predictions": [5, 5, 3, 5, 5, 5, 3, 4, 5, 5, 1, 4, 4, 4, 1, 2, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 1, 5, 1, 1, 1, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 5, 4, 5, 3, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 3, 5, 5, 1, 5, 5, 2, 5, 5, 5, 4, 5, 5, 4, 1, 1, 5, 5, 4, 5, 5, 1, 2, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 1, 2, 5, 5, 3, 1, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 1, 5, 5, 5, 1, 4, 3, 1, 5, 5, 4, 5, 5, 1, 5, 5, 3, 5, 4, 1, 1, 5, 5, 5, 4, 5, 5, 5, 5, 1, 2, 5, 5, 4, 5, 5, 1, 4, 5, 5, 5, 2, 3, 5, 5, 5, 5, 4, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 1, 1, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 1, 4, 4, 1, 2, 5, 1, 1, 4, 5, 5, 5, 1, 5, 4, 5, 1, 4, 4, 5, 4, 3, 4, 1, 5, 5, 5, 5, 5, 5, 3, 5, 1, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 4, 4, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5, 4, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 5, 5, 1, 4, 5, 1, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3

## Part F: Follow-up questions
Now that you have completed the coding portions, answer the following questions. Write your answers as if you are creating a technical report for your supervisor; limit your responses to **one or two paragraphs** (5-10 sentences per paragraph) in each question.

**Question 1 (6pts)**: In Part A (Data Cleaning), did you exclude any rows in the dataset? If you did, what are the exclusion criteria and how many rows were excluded? Did you remove or clean any column? Explain in sufficient details so that someone who reads your description could replicate the code from scratch.

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| Describe all the row-based operations. 	| 2pts 	|
| Describe all the column-based operations. 	| 2pts 	|
| Provide sufficiently detailed descriptions that can be used to reconstruct the code 	| 2pts 	|

**Your response**:

For column operations, first, I removed all the irrelevant features (colulmns) as suggested. Also, I implemented the suggested preprocessing function for both columns "review_headline" and "review_body". I also created a new column called "review" which concatenates the previous two columns.

For row operations, I checked there are no null values in the whole dataframe so I did not exlude any rows in the dataset up to this step.

**Question 2 (5pts)**: In Part B (Exploratory Data Analysis), which types of visualization did you employ (e.g., scatter plot, boxplot)? Why did you choose these types? Which columns or relationships between columns did you investigate? Summarize your findings.

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| List all of the graph types used along with their goal (e.g., “I used violinplot to see the distribution of data in column X,” instead of simply “I used violinplot”) 	| 1pt 	|
| List all the columns or relationships between columns that were examined. 	| 2pts 	|
| Describe the findings from the exploratory data analysis in layman’s terms, so that a reader not familiar with the dataset or your code can still understand them. 	| 2pts 	|

**Your response**:

I used boxplot to investigate the class imbalance problem of the label field "star_rating". I used violin plot to see the distribution of data in column "helpful_votes" and "total_votes". I also used scatterplot to investigate the relationship between the ratio of helpful votes and star rating.

In the boxplot, I found that the classes are extremely unbalanced, which provides direction for balancing the dataset in future steps. In the violin plot, I found several outliers based on total_votes and helpful_votes values, which I used to exclude some rows from the dataset. In the scatterplot, I examined the relationship between the ratio of helpful votes and star rating but did not find useful clue. This gives me the intuition that the numerial features might not be powerful predictors and the focus of feature engineering should be re-directed to text content.

I also did some EDA on the review text lengths. I found most of the texts have the length from 10 to 100 words. Only a few reviews are extremely long so I truncate every text to 100 words.

**Question 3 (6pts)**: In Part C (Feature Construction), which feature construction method(s) did you employ? Specify the associated parameters (for example, if you used Sklearn's TfIdf, what are your preprocessor, tokenizer and analyzer?).

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| List all of the feature construction steps, along with the state of the dataset before and after each step. 	| 3pts 	|
| In each step, specify the input parameters in sufficient details so that the original code can be reconstructed from your descriptions. 	| 3pts 	|

**Your response**:

I used Sklearn TF-IDF vectorizer and Count vectorizer as feature construction models. Before the feature construction step, each record in the dataset is the concatenation of review headline and review body text.

For TF-IDF vectorizer and Count vectorizer, I used "analyzer=str.split, tokenizer=str.split" as input parameters for both of them. They will transform the whole sentence (headline + body) into fixed length vectors. After feature constuction, the dataset will have the new shape of (number_examples, feature_dim).

**Question 4 (5pts)**: In Part D (Model Training and Evaluation), list all the models that you explored. How did you select the best model, or the best combination of models (if you used an ensemble method)? If you used train-test split or cross validation, describe them in detail (e.g., the training portion, the type of cross validation, etc.).

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| Name at least two explored models.	| 2pts 	|
| Provide sufficiently detailed descriptions of the model selection procedure that can be used to reconstruct the code. 	| 3pts 	|

**Your response**:

I explored Logistic Regression, SVM, and KNN.

Logistic regression gives the best test accuracy of 70% while the other two have significantly lowr perforamnce (~58% test accuracy). The detailed configuration of my Logistic Regression model are: C=0.99, multi_class="multinomial", solver='lbfgs' and all other parameters are default. The reason I only fine tune these two parameters is that the default version tends to overfit on the data. I set C=0.9 for the SVC model and fine tuned n_neighbors from 5 to 15 for the KNN model, but they do not seem to generate better results than the Logistic Regression model.

I also tried ensemble these models to make predictions. But since KNN and SVM do not predict well, the ensemble model is acutually worse than Logistic Regression Model alone.

For the best logistic regression model, before putting it into deployment, I fit it on the full dataset since we will test it on different data source

**Question 5 (3pts):** Did you complete Parts A-E in one pass, as an ordered sequence of steps, or was it necessary to revisit some earlier parts of your solution in order to achieve the desired outcome when working on a later part? If you proceeded sequentially, did you perform any data planning to identify the best steps for each part? If you had to revisit some parts of your solution, which did you revisit most, and why?

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| Mention whether you went through the parts A-E sequentially.	| 1pt 	|
| Describe your data planning if you proceeded sequentially, OR	| 2pts 	|
| Specify the part that you revisited most recently and the reason for doing so. | 2pts |
    
**Your response**:

I went through the parts A-E sequentially.

Yes, for data planning, according to my previous experience in NLP, Logistic Regression will be a pretty good baseline for text classification task. And for planning of vectorizer, from my persepctive, generally TFIDF will be better than Count Vectorizer.

**Question 6 (4pts)**: Did you identify any imbalance in the dataset? If you did, describe it here. Which method did you apply to address this imbalance? Describe the state of the dataset after you applied this method; was the identified imbalance mitigated or completely resolved?

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| Describe the imbalance in the dataset quantitatively.	| 2pts 	|
| Describe the steps used to mitigate the imbalance and the state of the dataset after applying these steps.	| 2pts 	|


**Your response**:

Yes, as discussed in the EDA part, the dataset is extremely unbalanced. The number of records for ratings 1 to 5 are 14136, 6373, 8483,17349, 53558 respectively. We need to over sample class 1 to 4 in the training set.

I used the RandomUpSampler oversampling algorithm in the imblearn library. We fit the upsampler AFTER the train test split on the training set. After upsampling, the number of records are all close to 53558 (not exactly the same since there are some random states).

The dataset imbalance problem is greatly mitigated after applying the techniques above

**Question 7 (5pts)**: Can you think of two other feature construction techniques that are different from what you implemented? How would they transform the given dataset? What are the advantages and disadvantages of the two proposed techniques, compared to the technique(s) used in your implementation?

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| Name the two proposed feature construction techniques (e.g., “Word embedding” or “Term-frequency matrix”).	| 1pts 	|
| Describe the dataset after applying each of the proposed techniques, e.g., what does the cell at row `i` and column `j` refer to?	| 2pts 	|
| Provide at least one advantage and one disadvantage for each of the two proposed techniques. | 2pts |

**Your response**:

The first model is Glove word embedding model. For glove word embedding model, I want to use the 300-dimension pretrained embedding model. I also specify the max length of each sentence to be 100 according to EDA on text lengths. After feature constuction, the dataset will have the new shape of (number_examples, 300 * max_length).

It is also interesting to use sentence level embedding model such as Google Universal Sentence Encoder and Facebook Infersent. These methods will transform the whole sentence to a fixed length vector of n_dim. After feature constuction, the dataset will have the new shape of (number_examples, n_dim).

The advantage of word embedding model is that we can do more customization on creating the embeddings. The disadvantage is that we need some extra coding work for concatenating word vectors for each sentence and fine tune max_length.

The advantage of sentence embedding model is that it is easy to use. We can just put in the sentence and get the fixed length embedding without doing any extra work. Correspondingly, the disadvantage is that we don't have much space for custimization.

**Question 8 (2pts)**: Which parts in the entire pipeline took the most development time? If you were to do a similar assignment in the future, how would you reduce the time taken for these parts of the solution?

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| Describe the time-consuming portions of the pipeline and the reasons for them being time-consuming.	| 1pt 	|
| Describe methods to reduce time in the future assignments.	| 1pt 	|


**Your response**:

The deployment part, especially debugging the score.py file part, takes me much time to finish.

At first I did not pay attention to the instruction that we will not be using inference_schema package this time and spend a lot of time figuring out the input and output schema. Next time to reduce time I need to review the insturctions more carefully to avoid wasting time.

**Question 9 (4pts)**: Are there differences in the language used in reviews with high ratings and those with low ratings? You can decide your own threshold for high vs. low and compare the review texts between the two groups. For example, did reviews with high ratings tend to contain more positive words, or have greater lengths?

**Grading rubric**:

| Rubric item 	| Score 	|
|-	|-	|
| Clearly define “low” and ‘high” reviews.	| 1pt 	|
| Clearly characterize one difference between the language used in low and high reviews in layman’s terms.	| 3pts 	|

**Your response**:


Yes. I define by rating score >= 3 as high review and others as low review.

I found that for shorter reviews with many postive words such as "good" have a higher probability of being a high rating review. I did not see such correlation between positive words and rating scores in longer text review. I speculate the reason might be that in the term frequency computation, positive words in shorter text will have more frequency compared to in longer text.